# Setup and Settings

Necessary imports and settings:

In [1]:
import torch, click, os
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import pandas as pd
import plotly.io as pio
import plotly.express as px
pio.renderers.default = "vscode"

We now select a word to analyze. Note that to visualize data, you must have already created the processed data file using 
```sh
python main.py --word [your_word]
```

Processed data will be loaded from `cache/[your_word].pt`. 

In [2]:
word = 'right'



'/home/andrew/polysemia'

# Visualization

Running the following code block will create an interactive 3D PCA plot of the data extracted for the chosen word.

In [5]:

# Load the cached data for this word and ouput the sample size
with open(os.path.join(os.getcwd(), f'cache/{word}.pt'), 'rb') as FILE:
    tensor, metadata = torch.load(FILE)

num_samples = len(metadata)
print(f'Number of samples: {num_samples}')

# Preform PCA on the embedded vectors
pca = PCA(n_components = 3)
transformed_data = pca.fit_transform(tensor.to('cpu').numpy())

# Format the dataframe for display purposes
df = pd.concat([metadata.reset_index(), pd.DataFrame(transformed_data, columns=['x','y','z'])], axis = 1)
df['size'] = [0.1] * num_samples

# Display data
fig = px.scatter_3d(
    df, 
    x = 'x', 
    y = 'y', 
    z = 'z', 
    hover_name = 'context', 
    hover_data = ['title', 'author', 'authoryearofbirth'], 
    size = 'size', 
    opacity = 0.1, 
    width = 800, 
    height = 800
    )

fig.show()

df

Number of samples: 9580


,index,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,context,x,y,z,size
0,1,/home/andrew/polysemia/SPGC-tokens-2018-07-18/...,The Declaration of Independence of the United ...,"Jefferson, Thomas",1743.0,1826.0,['en'],604.0,"{'United States -- History -- Revolution, 1775...",Text,...votes but because it is RIGHT if a free soc...,-5.245569,-1.138656,3.466327,0.1
1,1,/home/andrew/polysemia/SPGC-tokens-2018-07-18/...,The Declaration of Independence of the United ...,"Jefferson, Thomas",1743.0,1826.0,['en'],604.0,"{'United States -- History -- Revolution, 1775...",Text,...cost of modern weapons both RIGHTly alarmed...,-0.314641,-1.327810,-3.529735,0.1
2,1,/home/andrew/polysemia/SPGC-tokens-2018-07-18/...,The Declaration of Independence of the United ...,"Jefferson, Thomas",1743.0,1826.0,['en'],604.0,"{'United States -- History -- Revolution, 1775...",Text,...and inventors the exclusive RIGHT to their ...,-4.032004,12.524797,-0.682451,0.1
3,3,/home/andrew/polysemia/SPGC-tokens-2018-07-18/...,John F. Kennedy's Inaugural Address,"Kennedy, John F. (John Fitzgerald)",1917.0,1963.0,['en'],28.0,{'Presidents -- United States -- Inaugural add...,Text,...votes but because it is RIGHT if a free soc...,-5.245574,-1.138568,3.466527,0.1
4,3,/home/andrew/polysemia/SPGC-tokens-2018-07-18/...,John F. Kennedy's Inaugural Address,"Kennedy, John F. (John Fitzgerald)",1917.0,1963.0,['en'],28.0,{'Presidents -- United States -- Inaugural add...,Text,...cost of modern weapons both RIGHTly alarmed...,-0.314644,-1.327800,-3.529758,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9575,262,/home/andrew/polysemia/SPGC-tokens-2018-07-18/...,The Collected Poems of Rupert Brooke,"Brooke, Rupert",1887.0,1915.0,['en'],83.0,"{'English poetry', 'Poets, English -- 20th cen...",Text,...me to left and to RIGHT hunched figures and...,6.261596,-0.130853,-1.063472,0.1
9576,262,/home/andrew/polysemia/SPGC-tokens-2018-07-18/...,The Collected Poems of Rupert Brooke,"Brooke, Rupert",1887.0,1915.0,['en'],83.0,"{'English poetry', 'Poets, English -- 20th cen...",Text,...so light we were so RIGHT we were so fair f...,-4.703043,-3.315282,0.346999,0.1
9577,262,/home/andrew/polysemia/SPGC-tokens-2018-07-18/...,The Collected Poems of Rupert Brooke,"Brooke, Rupert",1887.0,1915.0,['en'],83.0,"{'English poetry', 'Poets, English -- 20th cen...",Text,...the sonnets which are RIGHTly given pride o...,-1.741812,-2.186448,0.913653,0.1
9578,262,/home/andrew/polysemia/SPGC-tokens-2018-07-18/...,The Collected Poems of Rupert Brooke,"Brooke, Rupert",1887.0,1915.0,['en'],83.0,"{'English poetry', 'Poets, English -- 20th cen...",Text,...with absolute conviction of the RIGHTness o...,-1.415003,1.465187,5.294522,0.1
